# Gemini Pro - Simple RAG with Vertex AI Search as a grounding service.

* This notebook explains how to use grounding service in Gemini Pro.
* **To verify the chunking options of data in a data store** 

In [59]:
%pip install --upgrade --quiet google-cloud-aiplatform
%pip install --upgrade --quiet google-cloud-discoveryengine
from IPython.display import display, Markdown

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [60]:
import os
from google.cloud import storage
from dotenv import load_dotenv, set_key
from google.oauth2 import service_account
import google.oauth2.credentials

# Load Google API Key from .env file
env_path = '.env'
load_dotenv(env_path)
api_key = os.getenv("GOOGLE_API_KEY")
if not api_key:
    raise ValueError("Failed to load API key. Please set GOOGLE_API_KEY in the .env file.")

# Service account key file path and project settings
KEY_PATH = "./pablo-test-425702-22d29fa73af8.json"
PROJECT_ID = "pablo-test-425702"
GS_LOCATION = "asia-northeast3"
MODEL_NAME="gemini-1.5-flash"
VERTEX_AI_LOCATION="us-central1"
try:
    storage_client = storage.Client.from_service_account_json(KEY_PATH)
    print("Google Cloud Storage client initialized")
except Exception as e:
    print(f"Google Cloud Storage client error: {e}")

credentials = service_account.Credentials.from_service_account_file(
    KEY_PATH, 
    scopes=['https://www.googleapis.com/auth/cloud-platform']
)




Google Cloud Storage client initialized


In [61]:
import vertexai
from vertexai.generative_models import GenerativeModel, Part, Tool
import vertexai.generative_models as generative_models

# Grounding service is still in preview.
from vertexai.preview.generative_models import grounding

# Initalizate the current vertex AI execution environment.
vertexai.init(project=PROJECT_ID, location=VERTEX_AI_LOCATION)
# Access to the generative model.
model = GenerativeModel(MODEL_NAME)

In [62]:
SEARCH_URL = "https://discoveryengine.googleapis.com/v1alpha/projects/151473909705/locations/global/collections/default_collection/dataStores/428e9604-0112-4316-a30e-cdd76f902169/servingConfigs/default_search:search"

In [63]:
import vertexai
import google
import google.oauth2.credentials
from google.auth import compute_engine
import google.auth.transport.requests
import requests
import json
import os

gcloud_path = "/Users/pablo/Desktop/workspaces/RAG/google-cloud-sdk/bin/gcloud"  # `which gcloud` 명령어로 확인한 경로
stream = os.popen(f'{gcloud_path} auth print-access-token')
credential_token = stream.read().strip()
print(credential_token)

ya29.a0AXooCgv52tYuQWSXFN9JMJfvhc47B05dZjM3CMF043bMpegrcqB46WxZF1kBiFDCf6KlbtoxYLmRxEdYGsIJSrD1mzPNA_lh2bk_BedZqp2ovEQ2iUHRuBeilBaA8618wTAJCodJ-c_DDbD2l123YjbUL8_fdsr77rUsidmR4PX9aCgYKAbkSARMSFQHGX2Mij72JNsRob2Ueci0UMF7AHQ0179


In [64]:
import requests
import html

def retrieve_vertex_ai_search(question:str, search_url:str, page_size:int)->str:

  """ retrieve information from enterprise search ( discovery engine )"""

  # Create a credentials token to call a REST API
  headers = {
      "Authorization": "Bearer "+ credential_token,
      "Content-Type": "application/json"
  }

  query_dic ={
      "query": question,
      "page_size": str(page_size),
      "offset": 0,
      "contentSearchSpec":{
            "searchResultMode" : "CHUNKS",
            "chunkSpec" : {
                "numPreviousChunks" : 2,
                "numNextChunks" : 2
            }
      },
  }

  data = json.dumps(query_dic)
  data=data.encode("utf8")

  response = requests.post(search_url,headers=headers, data=data)

  print(response.text)
  decoded_text = html.unescape(response.text)
  print(decoded_text)

  
  
  
  return response.text

In [65]:
question = "LS의 발행 주식수와, 미국의 2035년 필요 Cable수요에 대해 알려주세요."

page_size = 2

searched_ctx = retrieve_vertex_ai_search(question, SEARCH_URL, page_size)

{
  "results": [
    {
      "chunk": {
        "name": "projects/151473909705/locations/global/collections/default_collection/dataStores/428e9604-0112-4316-a30e-cdd76f902169/branches/0/documents/2d8dd418d9714276b2812d4d7a98228d/chunks/c1",
        "id": "c1",
        "content": "Korea | 철강금속 | 2024.07.11LS\n(006260)투자의견\nBUY(유지)\n목표주가\n190,000 원(유지)\n현재주가\n156,700 원(07/10)\n시가총액\n5,046(십억원)기다리고 기다렸던\n미국 해저케이블 공장철강/금속 이유진_ 02)368-6141_ eugenelee@eugenefn.com 7/10, 미국 생산시설 투자 공시\n- LS Green Link USA(LS전선 100%, 2023/10 신설) 해저케이블 공장 건설을 위해 LS전선이 약 9,418억\n원(6억 8,475만 달러)의 투자를 결의하는 공시 발표\n- 규모는 매출액 기준 5,000~6,000억원(동해 해저케이블 4, 5동 합한 규모), 향후 증설 위한 1개 라인\n추가 가능. 완공은 ‘27년 목표, 위치 Virginia\n 미국 = 해저케이블 공급 우위 시장\n- 미국 목표: 미국 정부는 ‘30년까지 30GW의 해상풍력 건설 목표 중. DOE가 2024/4월에 발표한 자료\n에 따르면 현재 미국 내 2024/4월까지 건설 중인 용량은 5GW, 건설 승인된 용량은 10GW, 추가적\n으로 5~10GW가 FID 대기 중. DOE와 NREL은 미국 내 해저케이블 생산시설이 있지만 수요를 충족하\n고 있지 못하고 있으며, ‘30년까지 30GW 달성을 위해서는 2,100개의 터빈, 10,000km이상의 내/외부\n해저케이블이 필요하다 언급\n- 미국 내 생산 업체: 미국

In [68]:
def parse_chunks(response_text:str)->dict:

    """Parse response to build a conext to be sent to LLM"""

    dict_results = json.loads(response_text)

    index = 0
    search_results = {}

    if dict_results.get('results'):

        for result in dict_results['results']:

            item = {}

            chunk = result['chunk']
            item['title'] = chunk['documentMetadata']['title']
            item['uri'] = chunk['documentMetadata']['uri']
            item['content'] = chunk['content']
            # print(chunk['chunkMetadata'].keys())
            # Chunks appear starting from those closest to the current Contents.
            
            if 'previousChunks' in chunk['chunkMetadata']:
                p_chunks = chunk['chunkMetadata']['previousChunks']
                for p_chunk in p_chunks:
                    item['content'] = p_chunk['content'] +"\n"+ item['content']

            n_chunks = chunk['chunkMetadata']['nextChunks']
            for n_chunk in n_chunks:
                item['content'] = item['content'] +"\n"+ n_chunk['content']

            search_results[f'results-{index}'] = item
            index = index+1

    return search_results

context = parse_chunks(searched_ctx)

In [69]:
def generate(query:str):
    """
    Generate a response from the model.

    query :
      query to be sent to the model

    Returns:
      The generated response.

    """

    # Set model parameter : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/send-multimodal-prompts#set_model_parameters
    generation_config = {
        "max_output_tokens": 8192,
        "temperature": 1,
        "top_p": 0.95,
    }

    # Configure satey setting : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/configure-safety-attributes
    # Refer to the link to remove : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/configure-safety-attributes#how_to_remove_automated_response_blocking_for_select_safety_attributes
    safety_settings = {
        generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    }

    responses = model.generate_content(
        [query],
        generation_config=generation_config,
        safety_settings=safety_settings,
        stream=False,
    )

    return responses.text

In [70]:
from time import perf_counter

t1_start = perf_counter()

prompt = f"""

  당신은 주식을 상담해주는 최고의 주식 전문가 입니다.
  아래 Question 에 대해서 반드시 Context에 있는 개별 내용을 기반으로 단계적으로 추론해서 근거를 설명하고 답변해주세요.
  Context : {context}
  Question : {question}
  """

outcome = generate(prompt)

t1_end  = perf_counter()
print(f"Time : {t1_end - t1_start} seconds\n\n")

display(Markdown(outcome))


Time : 5.899772249991656 seconds




## LS의 발행 주식수와 미국의 2035년 필요 Cable 수요에 대한 답변입니다.

**1. LS의 발행 주식수:**

* Context의 `results-0` 데이터를 살펴보면, LS의 발행 주식수는 **32,200천주**입니다. 
* 이 정보는 `results-0`의 `content` 부분에서 "발행주식수32,200천주"라고 명시되어 있습니다.

**2. 미국의 2035년 필요 Cable 수요:**

* Context에서 2035년 필요 Cable 수요에 대한 직접적인 정보는 제공되지 않습니다. 
* 다만, Context의 `results-0` 데이터에서 "미국 정부는 ‘30년까지 30GW의 해상풍력 건설 목표 중"이라고 언급되어 있습니다. 
* 또한, "DOE와 NREL은 미국 내 해저케이블 생산시설이 있지만 수요를 충족하고 있지 못하고 있으며, ‘30년까지 30GW 달성을 위해서는 2,100개의 터빈, 10,000km이상의 내/외부 해저케이블이 필요하다"라고 명시되어 있습니다. 
* 따라서, 2030년까지 10,000km 이상의 해저케이블이 필요하며, 2035년에는 이보다 더 많은 수요가 발생할 가능성이 높다고 추론할 수 있습니다. 
* 하지만 Context에는 2035년 필요 Cable 수요에 대한 명확한 수치가 없으므로, 정확한 답변은 어렵습니다.

**결론:**

* LS의 발행 주식수는 32,200천주입니다. 
* 미국의 2035년 필요 Cable 수요는 Context에서 명확하게 제시되지 않았으므로, 2030년까지 10,000km 이상의 해저케이블 수요가 발생할 것으로 예상되며, 2035년에는 이보다 더 많은 수요가 있을 가능성이 높다고 추론할 수 있습니다. 
